# 1. Importation des librairies

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Chargement des données

In [ ]:
###Chargement des données dans un Dataframe Pandas
spaceship_titanic_train = pd.read_csv('train.csv')
print(f"La taille (Lignes et colonnes) de l'ensemble des données d'entraînement : {spaceship_titanic_train.shape}")

In [ ]:
###Affichons les 5 premiers exemples
###dataset_df.head() ou
spaceship_titanic_train.head(5)

# 3. Explorations et visualisations de nos données

In [ ]:
###On peut constater que la moyenne d'âge est de 28 (28.827 ~ 28), dont le maximum est 79 ans.
###Avec 8514 exemples
spaceship_titanic_train.describe()

In [ ]:
###Voir le DataFrame
spaceship_titanic_train

In [ ]:
###Les nombre de passagers transportés (True) et non transportés (False) sont presque identiques
###Dans nos données d'entraînement
ax = sns.countplot(spaceship_titanic_train, x='Transported')
ax.set_title('Nombre de passagers transportés et non transportés')
ax.set_xlabel('Passagers transportés')
ax.set_ylabel('Nombre')

In [ ]:
###Distribution de l'âge par rapport
###aux passagers transportés (True) et non transportés (False)
###
###On verra que les passagers de :
###0-18 ans étaient plus susceptibles d'être transportés que de ne pas l'être.
###18-25 ans étaient moins susceptibles d'être transportés que de ne pas l'être.
###Les plus de 25 ans ont à peu près autant de chances d'être transportés que de ne pas l'être.
sns.histplot(data=spaceship_titanic_train, x='Age', hue='Transported', binwidth=1, kde=True)
plt.title('Distribution de l\'âge')

In [ ]:
###Les informations sur le DataFrame,
###Notamment le type d'index et les colonnes, les valeurs non nulles
###Et l'utilisation de la mémoire.
spaceship_titanic_train.info()

# 4. Préparation de nos données

In [ ]:
###Suppression des colonnes non importantes pour nos données d'entraînement
spaceship_titanic_train = spaceship_titanic_train.drop(['PassengerId', 'Name'], axis=1)
spaceship_titanic_train.head(2)

In [ ]:
###Voir le nombre de valeurs manquantes dans nos données
spaceship_titanic_train.isnull().sum().sort_values(ascending=False)

In [ ]:
# df = spaceship_titanic_train.copy()
# numeric_columns = df.select_dtypes(include=['number']).columns
# nrows = len(numeric_columns)
# ncols = 1

# _, axes = plt.subplots(nrows, ncols, figsize=(8, 6*nrows))

# for i, col in enumerate(numeric_columns):
#     ax = axes[i]
#     col_random = col + '_random'
#     df[col_random]=df[col]
#     random_sample = df[col].dropna().sample(df[col].isnull().sum(), random_state=0)
#     random_sample.index = df[df[col].isnull()].index
#     df.loc[df[col].isnull(), col_random] = random_sample

#     df[col].plot(kind='kde', ax=ax, label=f"{col} original", color='blue')
#     df[col_random].plot(kind='kde', ax=ax, label=f"{col} random", color='red')

#     ax.set_title(f"Distribution de {col}")
#     ax.legend()
#     ax.set_xlim(df[col].min(), df[col].max())
#     ax.set_ylim(0, None)

# plt.show()

In [ ]:
# df = spaceship_titanic_train.copy()
# numeric_columns = df.select_dtypes(include=['number']).columns
# print(numeric_columns)
# nrows = len(numeric_columns)
# ncols = 1
# _, axes = plt.subplots(nrows, ncols, figsize=(8, 6*nrows))

# for i, col in enumerate(numeric_columns):
#     ax = axes[i]
#     col_median = col + '_median'
#     df[col_median] = df[col].fillna(df[col].median())

#     df[col].plot(kind='kde', ax=ax, label=f"{col} original", color='blue')
#     df[col_median].plot(kind='kde', ax=ax, label=f"{col} median", color='red')

#     ax.set_title(f"Distribution de {col}")
#     ax.legend()
#     ax.set_xlim(df[col].min(), df[col].max())
#     ax.set_ylim(0, None)

# plt.show()

In [ ]:
df = spaceship_titanic_train.copy()
df.isnull().sum().sort_values(ascending=False)

In [ ]:
target_col = 'Transported'
for col in df.columns:
    if col != target_col:
        random_sample = df[col].dropna().sample(df[col].isnull().sum(), random_state=0)
        random_sample.index = df[df[col].isnull()].index
        df.loc[df[col].isnull(), col] = random_sample

df.isnull().sum().sort_values(ascending=False)

In [ ]:
# df.duplicated().sum()
# df.drop_duplicates()

In [ ]:
###On convertit les colonnes boolean en entier
bool_cols = [target_col, 'CryoSleep', 'VIP']
for col in bool_cols:
    df[col] = df[col].astype(int)

df.head(2)

In [ ]:
###On récupère les variables catégorielles de nos données
categorical_features = []
for col in df.select_dtypes(object).columns:
    categorical_features.append(col)

categorical_features

In [ ]:
###Encoder les variables catégorielles via Pandas
# df = pd.get_dummies(df, drop_first=True)
# df.head(2)


###Encoder les variables catégorielles via Scikit-Learn
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

one_hot = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
transformed_data = ColumnTransformer(
    [('one_hot', one_hot, categorical_features)],
    remainder='passthrough'
)

transformed_data.fit_transform(df)

In [ ]:
###Création des données de test et d'entraînement
from sklearn.model_selection import train_test_split
X = df.drop(columns=[target_col])
y = df[target_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

### Voir la taille des données subdivisées
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 5. Contruction et entraînement de plusieurs modèles

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
# from sklearn.metrics import accuracy_score
# knn = KNeighborsClassifier(
#     n_neighbors=100,
#     # weights='distance',
#     # n_jobs=100
# )
# knn.fit(X_train, y_train)
# y_pred_knn = knn.predict(X_test)
# from sklearn.metrics import accuracy_score
# accuracy_knn = accuracy_score(y_test, y_pred_knn)
# accuracy_knn

In [ ]:
# rf = RandomForestClassifier(
#     n_estimators=100,
#     n_jobs=3,
#     random_state=0
# )
# rf.fit(X_train, y_train)
# y_pred_rf = rf.predict(X_test)
# accuracy_rf = accuracy_score(y_test, y_pred_rf)
# accuracy_rf

In [ ]:
###Definir plusieurs modèles avec ces hyperparamètres
models = [
    # (
    #     'KNeighborsClassifier',
    #     KNeighborsClassifier(),
    #     {
    #         'kneighborsclassifier__n_neighbors': [5, 10, 50, 100],
    #         'kneighborsclassifier__weights': ['uniform', "distance"],
    #         'kneighborsclassifier__n_jobs': [1, 2]
    #     }
    # ),
    # (
    #     'RandomForestClassifier',
    #     RandomForestClassifier(),
    #     {
    #         'randomforestclassifier__criterion': ['gini', 'entropy'],
    #         'randomforestclassifier__n_estimators': [100, 200, 300],
    #         'randomforestclassifier__max_features': ['sqrt', 'log2'],
    #         'randomforestclassifier__max_depth': [None]
    #     }
    # ),
    # (
    #     'LogisticRegression',
    #     LogisticRegression(),
    #     {
    #         'penalty': [None, 'l1', 'l2', 'elasticnet'],
    #         'solver': ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    #         'max_iter': [1000, 2000, 3000]
    #     }
    # ),
    # (
    #     'DecisionTreeClassifier',
    #     DecisionTreeClassifier(),
    #     {
    #         'decisiontreeclassifier__criterion': ['gini', 'entropy'],
    #         'decisiontreeclassifier__splitter': ['best', 'random'],
    #         'decisiontreeclassifier__max_features': [None, 'sqrt', 'log2'],
    #         'decisiontreeclassifier__max_depth': [None, 10, 20, 30, 40, 50]
    #     }
    # ),
    (
        'SVC',
        SVC(),
        {
            'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
            'svc__decision_function_shape': ['ovo', 'ovr'],
            'svc__break_ties': [False, True]
        }
    ),
]

In [ ]:
models_score_results = []
for name, model, parameters in models:
    _pipeline = make_pipeline(transformed_data, model)
    print(name)
    print(_pipeline.get_params().keys())
    gridSearchCV = GridSearchCV(_pipeline, parameters, verbose=3)
    gridSearchCV.fit(X_train, y_train)
    models_score_results.append((name, gridSearchCV.best_params_, gridSearchCV.best_score_))
    print('\n')

In [ ]:
models_score_results

In [ ]:
# models_score_results = []
# for name, model, parameters in models:
#     _pipeline = make_pipeline(transformed_data, model)
#     print(name)
#     print(_pipeline.get_params().keys())
#     randomizedSearchCV = RandomizedSearchCV(_pipeline, parameters, verbose=3, n_iter=20)
#     randomizedSearchCV.fit(X_train, y_train)
#     models_score_results.append((name, randomizedSearchCV.best_params_, randomizedSearchCV.best_score_))
#     print('\n')

In [ ]:
# models_score_results